# Clasificación utilizando `sklearn`: predicción del estado de un crédito

En este notebook vamos entrenar distintos modelos de clasificación para predecir el estado de un crédito utilizando este dataset disponible en Kaggle: https://www.kaggle.com/zaurbegiev/my-dataset

Echa un vistazo a las columnas disponibles para entender de qué información se dispone.

En primer lugar, cargamos las librerías necesarias y listamos los ficheros del directorio actual para comprobar que estamos bien situados:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir("./"))

Cargamos el fichero de datos `credit_train.csv` y mostramos las primeras filas:

In [ ]:
data = pd.read_csv('credit_train.csv')
data.head()

Como ya es habitual, mostramos información básica acerca de cada dataset utilizando las funciones `info()` y `describe()`:

In [ ]:
print("Dataset size: ", data.shape)
print('**'* 50)
data.info()
print('**'* 50)
data.describe()

## Preprocesado de datos

### Limpieza y comprobación de valores perdidos

En primer lugar, vemos que Las columnas `Loan ID`y `Customer ID` simplemente son para identificación y no se utilizan para entrenar los modelos, las eliminamos:

In [ ]:
data.drop(labels=['Loan ID', 'Customer ID'], axis=1, inplace=True)
print("Dataset size: ", data.shape)

Comprobamos cuántos valores nulos hay en cada columna:

In [ ]:
data.isnull().sum()

Creamos una función para devolver esta información ordenada y con el porcentaje sobre el total:

In [ ]:
def create_missing_values_table(df):
    missing_values = df.isnull().sum()

    missing_values_percent = 100 * df.isnull().sum() / len(df)

    table = pd.concat([missing_values, missing_values_percent], axis=1)

    table = table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'}
    )

    table = table[table.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)

    return table

create_missing_values_table(data)

Hay varias casuísticas. En primer lugar, vemos que existe una columna (`Months since last delinquent`) para la cual tenemos valores perdidos en más del 50% de las filas, por lo que la eliminamos.

In [ ]:
data.drop(columns = 'Months since last delinquent', axis=1, inplace=True)

print("Dataset size: ", data.shape)
display(create_missing_values_table(data))

Llaman la atención las 514 filas de varias columnas que tienen valores perdidos. ¿Será que todas esas filas solo tienen valores perdidos en todas las columnas? Lo comprobamos:

In [ ]:
data[data['Years of Credit History'].isnull() == True]

Curiosamente, son las 514 últimas filas del dataset, por lo que podemos eliminarlas:

In [ ]:
data.drop(data.tail(514).index, inplace=True)

print("Dataset size: ", data.shape)
display(create_missing_values_table(data))

Ahora tenemos 3 columnas (`Bankruptcies`, `Tax Liens` y `Maximum Open Credit`) con un número muy bajo de filas con valores perdidos sobre el total, por lo que podemos optar por eliminarlas:

In [ ]:
for i in data['Maximum Open Credit'][data['Maximum Open Credit'].isnull() == True].index:
    data.drop(labels=i, inplace=True)

for i in data['Tax Liens'][data['Tax Liens'].isnull() == True].index:
    data.drop(labels=i, inplace=True)

for i in data['Bankruptcies'][data['Bankruptcies'].isnull() == True].index:
    data.drop(labels=i, inplace=True)

print("Dataset size: ", data.shape)
display(create_missing_values_table(data))

Dos de las columnas que quedan son numéricas (`Credit Score` y `Annual Income`), podemos utilizar la estrategia de rellenar con el valor medio:

In [ ]:
data.fillna(data.mean(), inplace=True)

print("Dataset size: ", data.shape)
display(create_missing_values_table(data))

Y finalmente, nos queda la columna `Years in current job`. Veamos cuál es el valor más frecuente:

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data['Years in current job'])

Por lo que, tal y como vimos la semana pasada, podemos asignar el valor más frecuente `10+ years` a los valores perdidos. En el notebook de regresión lo hacíamos utilizando un `SimpleInputer`, aquí podemos hacerlo directamente con `fillna`:

In [ ]:
data.fillna('10+ years', inplace=True)

print("Dataset size: ", data.shape)
display(create_missing_values_table(data))

## Valores duplicados

Una función interesante es `drop_duplicates`, que permite eliminar filas redundandes. Si sospechamos que puede pasar esto en nuestro dataset, aplicamos esta función y vemos los cambios en el tamaño del dataset:

In [ ]:
print("Dataset size: ", data.shape)

data.drop_duplicates(inplace = True)

print("Dataset size without duplicates: ", data.shape)

Como vemos, había un número importante de filas duplicadas, así que nos quedamos con la versión limpia.

## Variables categóricas

En el dataset tenemos algunas variables categóricas, que es necesario convertir a variables numéricas para poder entrenar los modelos de clasificación.

In [ ]:
plt.figure(figsize=(20,8))
sns.countplot(data['Term'])

plt.figure(figsize=(20,8))
sns.countplot(data['Years in current job'])

plt.figure(figsize=(20,8))
sns.countplot(data['Home Ownership'])

plt.figure(figsize=(20,8))
sns.countplot(data['Purpose'])

Utilizaremos la *one-hot encoding* para transformar estas variables.

In [ ]:
categorical_subset = data[['Term', 'Years in current job', 'Home Ownership', 'Purpose']]

categorical_subset = pd.get_dummies(categorical_subset)

display(categorical_subset)

Dado que la columna `Term` solo tiene dos valores, con el valor de una columna sabemos el de la otra, por lo que podemos eliminar una de ellas. Eliminamos una de ellas y concatenamos la codificación *one-hot* al dataset original, tras haber eliminado las columnas originales:

In [ ]:
categorical_subset.drop(labels=['Term_Long Term'], axis=1, inplace=True)

data.drop(labels=['Term', 'Years in current job', 'Home Ownership', 'Purpose'], axis=1, inplace=True)
data = pd.concat([data, categorical_subset], axis = 1)

print("Dataset size: ", data.shape)

## Visualización de datos

En este apartado puedes aplicar cualquier técnica de visualización para analizar los datos de una o varias columnas en función de su tipo.

Nos fijaremos en la distribución de la columna que queremos predecir:

In [ ]:
display(data.shape)
display(data['Loan Status'].unique())
display(data['Loan Status'].value_counts())

plt.figure(figsize=(20,8))

sns.countplot(data['Loan Status'])

Como puedes ver, se trata de un dataset desbalanceado ya que tenemos casi un 75% de muestras `Fully paid`.

Esto es importante, porque un modelo que clasificase todas las muestras como `Fully paid`
obtendría una tasa de aciertos del 75%.

En este notebook usaremos el dataset desbalanceado, pero aquí están algunos recursos sobre estrategias para lidiar con este problema que puedes aplicar una vez terminado el notebook:
- https://www.kdnuggets.com/2017/06/7-techniques-handle-imbalanced-data.html
- https://elitedatascience.com/imbalanced-classes

Utilizaremos una métrica que permita tener en cuenta el desbalanceo como puede ser el F1 score (https://en.wikipedia.org/wiki/F-score).

## Clasificación

### División en Train y Validation

Cargamos las librerías para este apartado del notebook y separamos el dataset disponible en `X`, un dataframe con las variables predictoras, e `Y`, un dataframe con la variable objetivo.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

random_state = 2020

data.ml = data

print('Tamaño del dataset de entrenamiento (muestras x variables):', data.ml.shape)

X = data.ml.drop(columns='Loan Status')
Y = pd.DataFrame(data.ml['Loan Status'])

Recodificamos la variable de salida a 0 y 1. Esto evita problemas con ciertos modelos y métricas que requieren de este tipo de codificación.

*Importante*: a la hora de calcular ciertas métricas como `precision` o `recall`, debes saber cuál es la clase positiva (1) y negativa (0) para poder interpretar los resultados.

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
Y_binary = LabelEncoder().fit_transform(Y)

Dividiremos el dataset de entrenamiento en tres subcojuntos: 60% de datos para `train`, 20% `validation` y 20% para `test`. Primero dividimos en 80% para `train` (que podemos utilizar para una validación cruzada o dividirlo para una validación *holdout*) y 20% para `test`. El dataset `train` volvemos a dividirlo en dos, de manera que tengamos un 60% para `train` y un 20% para `validation`.

Utilizamos `stratify` para que se mantenga la distribución de la clase que queremos predecir en las particiones.

In [ ]:
Y = Y_binary

train_ratio = 0.60
test_ratio = 0.20
validation_ratio = 0.20

X_train_val, X_test, Y_train_val, Y_test = train_test_split(
    X, Y,
    test_size=test_ratio,
    stratify=Y,
    random_state=2020
)

X_train, X_val, Y_train, Y_val = train_test_split(
    X_train_val, Y_train_val, 
    test_size=validation_ratio/(test_ratio+train_ratio),
    stratify=Y_train_val,
    random_state=2020
)

A continuación:
- Entrenaremos varios modelos utilizando validación *holdout* y validación cruzada. El dataset `test` quedará reservado hasta el final.
- Buscaremos los mejores valores de parámetros de varios modelos utilizando validación cruzada.
- Compararemos los mejores modelos y escogeremos uno.
- Probaremos el modelo escogido en el dataset `test`.

## Entrenamiento de un modelo y validación *holdout*

Empezamos por entrenar un árbol de decisión, disponible en `DecisionTreeClassifier` (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Utilizamos la partición `train` (60%) para entrenar y `validation` (20%) para probar el modelo.

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = random_state)
dt.fit(X_train, Y_train)

dt_prediction = dt.predict(X_val)
print('Decision Tree accuracy = ', metrics.accuracy_score(dt_prediction, Y_val))

El árbol de decisión se puede visualizar de varias maneras, tal y como se explica aquí: https://mljar.com/blog/visualize-decision-tree/
        
    
Visualizamos las reglas en modo texto, que es una de las maneras más sencillas. Esto puede tardar un rato, así que ve echándole un vistazo al siguiente apartado.

*Disclaimer*: intenté visualizarlas gráficamente pero daba bastantes problemas no sé por qué, así que si queréis pelearos con esto adelante ;-)

In [ ]:
from sklearn import tree

text_representation = tree.export_text(dt, feature_names = data.columns.drop('Loan Status').tolist())
print(text_representation)

## Entrenamiento de un modelo  y validación cruzada

Otra manera de evaluar el rendimiento de un modelo es la *K-Fold Cross-Validation*. En esta página de `sklearn` puedes encontrar más información sobre ello: https://scikit-learn.org/stable/modules/cross_validation.html

La manera más sencilla es utilizar `cross_val_score`, que permite evaluar una sola métrica. En este ejemplo validaremos el rendimiento de un árbol de decisión utilizando esta técnica para aprender a usar el API. En este caso, utilizamos el 80% de los datos disponible en `X_train_val` (recuerda: es importante es no usar el `test` hasta tener escogido un modelo).

In [ ]:
from sklearn.model_selection import cross_val_score

dt = DecisionTreeClassifier(random_state = random_state)

scores = cross_val_score(estimator = dt, X = X_train_val, y = Y_train_val, cv = 5)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Por defecto, el `score` que se calcula es la tasa de aciertos (`accuracy`). Se puede utilizar otra métrica de evaluación (definidas en https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter), en función de las necesidades y de si el dataset está balanceado o no.

In [ ]:
dt = DecisionTreeClassifier(random_state = random_state)

scores = cross_val_score(estimator = dt, X = X_train_val, y = Y_train_val, cv = 5, scoring = 'f1')

print("F1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = cross_val_score(estimator = dt, X = X_train_val, y = Y_train_val, cv = 5, scoring = 'precision')

print("Precision: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Sin embargo, este enfoque no es práctico si queremos evaluar múltiples métricas a la vez. Para ello, se utiliza la función `cross_validate`.

In [ ]:
from sklearn.model_selection import cross_validate

scoring = ['precision', 'accuracy', 'f1']
dt = DecisionTreeClassifier(random_state = random_state)
scores = cross_validate(dt, X_train_val, Y_train_val, scoring=scoring)

display(sorted(scores.keys()))

display(scores['test_f1'])

Entre las métricas de evaluación verás que algunas tienen versiones *macro* y *micro*. Echa un vistazo a este post para entender en qué consiste: http://rushdishams.blogspot.com/2011/08/micro-and-macro-average-of-precision.html

## Ejercicio: variar los parámetros del árbol de decisión

Como ves, el rendimiento del árbol está por debajo del 75% que obtendríamos si predijésemos la clase mayoritaria. Echa un vistazo a los parámetros del árbol (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) y entrena distintos modelos variando algún parámetro para ver si consigues mejorar el rendimiento.

In [ ]:
## Añade parámetros a DecisionTreeClassifier y prueba distintos valores
dt = DecisionTreeClassifier(random_state = random_state)
dt.fit(X_train, Y_train)

dt_prediction = dt.predict(X_val)
print('Decision Tree accuracy = ', metrics.accuracy_score(dt_prediction, Y_val))

## Búsqueda de los mejores parámetros

En el ejercicio anterior intentábamos buscar parámetros que mejorasen el rendimiento del árbol. Esto se puede hacer de manera automática utilizando `GridSearchCV` (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Echa un vistazo a la documentación de `sklearn` disponible en esta otra página (https://scikit-learn.org/stable/modules/grid_search.html) antes de ejecutar el siguiente código. Presta atención al `refit` y lo que significa.

En este ejemplo creamos un *grid* con los valores de dos parámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV

dt_parameters = {
    'criterion':('gini', 'entropy'),
    'max_depth': np.arange(1, 15, 4)
}

dt = DecisionTreeClassifier()
dt.gscv = GridSearchCV(dt, dt_parameters, cv=5, scoring='f1')
dt.gscv.fit(X_train_val,Y_train_val)

E imprimimos las variables `best_params_` y `cv_results_` para obtener el resultado de la mejor combinación:

In [ ]:
print(dt.gscv.best_params_)
print(dt.gscv.best_estimator_)
print(dt.gscv.best_score_)

Sorprendentemente, el mejor resultado se obtiene con una profundidad de 1, lo cual nos puede hacer pensar que quizá el árbol de decisión no tenga la suficiente capacidad de generalización para este dataset concreto. 

Si quisiéramos seleccionar el mejor modelo y utilizarlo, este estaría en `best_estimator_`. A modo de ejemplo, calculamos la tasa de aciertos del mejor modelo en los datos de validación:

In [ ]:
dt_prediction = dt.gscv.best_estimator_.predict(X_val)
print('Best Decision Tree accuracy = ', metrics.accuracy_score(dt_prediction, Y_val))

También podemos imprimir todos los valores de resultados obtenidos en la validación cruzada:

In [ ]:
print(dt.gscv.cv_results_)

## Ejercicios

Repite los pasos anteriores para distintos modelos:
- Random Forest (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).
- KNN (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).
- SVM (https://scikit-learn.org/stable/modules/svm.html).
- Logistic Regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

Para cada modelo crea un apartado en el notebook y:
- Comienza probando un clasificador básico con los parámetros por defecto.
- Varía algún parámetro para ver cómo cambia el rendimiento.
- Busca la mejor combinación de parámetros con `GridSearchCV`.

Algunos consideraciones:
- En el caso de la SVM necesitarás escalar los datos, algo que hicimos en notebooks anteriores. Fíjate en que en el ejemplo de SVM (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) se utiliza un `Pipeline`. Esto es necesario porque los datos se deben escalar utilizando la media y la desviación típica partición de entrenamiento de cada fold, de lo contrario se produciría *data leakage* Mira el apartado de lecturas adicionales.
- En el caso del Random Forest, puedes intentar extraer la importancia de cada variable y hacer una representación gráfica.
- Si algún modelo tarda mucho en entrenarse (o en completar la búsqueda con `GridSearchCV`), puedes probar a reducir el tamaño del dataset (por ejemplo, eliminando todas las filas con valores perdidos al principio).

Finalmente, crea un apartado en el notebook para comparar el rendimiento de los distintos modelos (por ejemplo, escogiendo el que mejor F1-score tenga) para escoger el que considieres mejor y aplica este modelo al dataset `test` que tenemos reservado.

Si tienes tiempo y ganas, puedes probar otros modelos o alguna de las estrategias de balanceo (over/up-sampling).

## Lecturas adicionales

El *data leakage* se produce cuando de algún modo se utilizan datos (muestras) de test durante el proceso de entrenamiento. Si aplicamos *feature selection* antes de entrenar un modelo y utilizamos todas las muestras para ello, estaríamos ocasionando *data leakage*. En esta página (http://thatdatatho.com/2018/10/04/cross-validation-the-wrong-way-right-way-feature-selection/) puedes ver un ejemplo. En este ejemplo de `sklearn` (https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html), puedes ver un pipeline sencillo en que se aplica *Principal Component Analysis* para transformar las variables seguida de una regresión lógistica para hacer la clasificación.